In [0]:
from pyspark.sql.functions import *

### Bảng tổng hợp số phim theo năm và thể loại

In [0]:
df_titles = spark.table("netflix_catalog.silver.netflix_titles_with_category")

df_gold_movie_stats = df_titles.groupBy("release_year", "listed_in") \
    .count() \
    .withColumnRenamed("count", "num_movies")

df_gold_movie_stats.write.format("delta").mode("overwrite").saveAsTable("netflix_catalog.gold.movie_stats_year_category")


In [0]:
%sql
SELECT * FROM netflix_catalog.gold.movie_stats_year_category LIMIT 10

### Bảng top diễn viên xuất hiện nhiều phim

In [0]:
df_cast = spark.table("netflix_catalog.silver.netflix_cast")

df_top_cast = df_cast.groupBy("cast") \
    .count() \
    .orderBy(col("count").desc())

df_top_cast.write.format("delta").mode("overwrite").saveAsTable("netflix_catalog.gold.top_cast")

In [0]:
%sql
SELECT * FROM netflix_catalog.gold.top_cast LIMIT 10

### Bảng phân tích phim theo quốc gia

In [0]:
df_countries = spark.table("netflix_catalog.silver.netflix_countries")

df_country_stats = df_countries.groupBy("country").count()

df_country_stats.write.format("delta").mode("overwrite").saveAsTable("netflix_catalog.gold.country_stats")


In [0]:
%sql
SELECT * FROM netflix_catalog.gold.country_stats LIMIT 10

### Thống kê số lượng phim theo năm phát hành và rating

In [0]:
df_titles = spark.table("netflix_catalog.silver.netflix_titles_with_category")
df_fact_year_rating = df_titles.groupBy("release_year", "rating") \
    .count() \
    .withColumnRenamed("count", "num_titles")

df_fact_year_rating.write.format("delta").mode("overwrite") \
    .saveAsTable("netflix_catalog.gold.fact_year_rating")

In [0]:
%sql
SELECT * FROM netflix_catalog.gold.fact_year_rating LIMIT 10

### Thống kê số phim theo rating và thể loại (listed_in)

In [0]:
df_fact_rating_category = df_titles.groupBy("rating", "listed_in") \
    .count() \
    .withColumnRenamed("count", "num_titles")

df_fact_rating_category.write.format("delta").mode("overwrite") \
    .saveAsTable("netflix_catalog.gold.fact_rating_category")

In [0]:
%sql
SELECT * FROM netflix_catalog.gold.fact_rating_category LIMIT 10

### Thống kê trung bình thời lượng phim theo năm phát hành và loại

In [0]:
from pyspark.sql.functions import avg

df_fact_avg_duration = df_titles.groupBy("release_year", "type") \
    .agg(avg("duration_minutes").alias("avg_duration_minutes"))

df_fact_avg_duration.write.format("delta").mode("overwrite") \
    .saveAsTable("netflix_catalog.gold.fact_avg_duration")

In [0]:
%sql
SELECT * FROM netflix_catalog.gold.fact_avg_duration LIMIT 10